In [1]:
import sys

In [2]:
sys.path.append('/home/ec2-user/SageMaker/')
sys.path.append('/home/ec2-user/SageMaker/bokeh_interactive/')
sys.path.append('/home/ec2-user/SageMaker/bokeh_interactive/tools/')
sys.path.append('/home/ec2-user/SageMaker/bokeh_interactive/tools/wrangle_data.py')
sys.path.append("tools/wrangle_data.py")

In [3]:
from tools.wrangle_data import parse_community_data

In [4]:
import pandas as pd
import multiprocessing as mp
from multiprocessing import Pool
import pickle


In [5]:
mp.cpu_count()

8

In [6]:
df_community = pd.read_csv("data/Community_Context/nov18pub.dat", header=None)


works = [df_community.values.squeeze()[i] for i in range(len(df_community.values.squeeze()))]


In [7]:
pool = Pool(processes=4)

In [8]:
results = pool.map(parse_community_data, works)

In [9]:
df_results = pd.DataFrame(results)

In [10]:
df_results.shape

(143050, 8)

In [11]:
df_results.columns

Index(['interview_type', 'voting_type', 'registered_type', 'age_range',
       'year_of_interview', 'state_code', 'metropolitan_code', 'county_code'],
      dtype='object')

In [12]:
df_results.voting_type.value_counts()

-1    54301
 1    48337
 2    25606
-9    11626
-2     1800
-3     1380
Name: voting_type, dtype: int64

In [13]:
df_results.interview_type.value_counts()

 1    122744
 3     10141
 2      9642
 4       523
Name: interview_type, dtype: int64

In [14]:
df_results.age_range.value_counts()

 2    99942
 1    22352
-1    20306
 3      450
Name: age_range, dtype: int64

## Remove Unqualified Interviewees

In [34]:
df_results_filtered = df_results.loc[df_results.voting_type != '-1', :]

In [35]:
df_results_filtered.shape

(88749, 8)

In [36]:
df_results_filtered.head(2)

,interview_type,voting_type,registered_type,age_range,year_of_interview,state_code,metropolitan_code,county_code
0,1,2,2,2,2018,01,33860,000
1,1,2,1,2,2018,01,33860,000


In [37]:
df_results_filtered.year_of_interview.value_counts()

2018    88749
Name: year_of_interview, dtype: int64

## Map metropolitan_code

In [18]:
with open('data/Community_Context/mappers.pkl', 'rb') as f:
    mappers_all = pickle.load(f)

In [19]:
mappers_all.keys()

dict_keys(['mapper_metro', 'mapper_age_range', 'mapper_interview_type', 'mapper_state_code', 'mapper_registered_type'])

In [20]:
mapper_metro = mappers_all['mapper_metro']

In [21]:
set(df_results_filtered.metropolitan_code) - set(mapper_metro.keys())

set()

In [22]:
df_results_filtered.metropolitan_code = list(df_results_filtered.metropolitan_code.map(mapper_metro))

C:\Users\mekho\Documents\PycharmProjects\venvs\LinkedIn_Quiz_env\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [23]:
df_results_filtered.metropolitan_code

0                            Montgomery, AL
1                            Montgomery, AL
2                            Montgomery, AL
9                 Daphne-Fairhope-Foley, AL
10                Daphne-Fairhope-Foley, AL
                        ...                
143022    NOT IDENTIFIED OR NONMETROPOLITAN
143029    NOT IDENTIFIED OR NONMETROPOLITAN
143035    NOT IDENTIFIED OR NONMETROPOLITAN
143036    NOT IDENTIFIED OR NONMETROPOLITAN
143038    NOT IDENTIFIED OR NONMETROPOLITAN
Name: metropolitan_code, Length: 40412, dtype: object

In [24]:
set(df_results_filtered.metropolitan_code) - set(mapper_metro.values())

set()

## Map registered_type

In [25]:
mapper_registered_type = mappers_all['mapper_registered_type']

In [26]:
set(df_results_filtered.registered_type) - set(mapper_registered_type.keys())

set()

In [27]:
df_results_filtered.registered_type = list(df_results_filtered.registered_type.map(mapper_registered_type))

In [28]:
df_results_filtered.registered_type

0                  no
1                 yes
2                 yes
9                 yes
10                yes
             ...     
143022    no_response
143029    no_response
143035    no_response
143036    no_response
143038    no_response
Name: registered_type, Length: 40412, dtype: object

In [29]:
df_results_filtered.registered_type.value_counts()

no             13026
yes            12012
no_response    11743
not_know        2335
refused         1296
Name: registered_type, dtype: int64

In [31]:
df_results_filtered.voting_type.value_counts()

 2    25606
-9    11626
-2     1800
-3     1380
Name: voting_type, dtype: int64

In [30]:
# df_results.to_csv('data/')